In [1]:
import os
import sys
import pandas as pd 
import numpy as np 
import boto3
import pymysql
import mysql.connector
from dotenv import load_dotenv
load_dotenv()

True

In [228]:
class DatabaseConnection:

    def __init__(self, host, user, password, database):
        try:
            self.connection = mysql.connector.connect(
                host=host,
                user=user,
                password=password,
                database=database
            )
            self.cursor = self.connection.cursor()

        except Exception as e:
            print(e)
    def execute_query(self, query):
        """
        Executes a SQL query and returns the results
        
        Args:
            query (str): SQL query to execute.
            
        Returns: query results.
        """
        
        try:
            self.cursor.execute(query)
            self.connection.commit()
            results = self.cursor.fetchall()  # Fetch all results
            return results
        except Exception as e:
            return f"Error executing query: {str(e)}"
        
        
    def execute_query_to_dataframe(self,query):
        """
        Executes a SQL query and returns the results as a pandas DataFrame.

        Args:
            query (str): SQL query to execute.

        Returns:
            pd.DataFrame: DataFrame containing the query results.
        """
        try:
            self.cursor.execute(query)
            results = self.cursor.fetchall()  # Fetch all results
            columns = self.cursor.column_names
            df = pd.DataFrame(results, columns=columns)
            return df

        except Exception as e:
                return f"Error executing query: {str(e)}"

        

    def insert_data_batch(self, query, data, BATCH_SIZE):
        """
        

        Args:
            query (str): This method will be take query arguments as string
            data (take data as tuple): pass data as tuple
            BATCH_SIZE (int): Give batch size acording to your needs.
        """
        try:
            
            for i in range(0, len(data), BATCH_SIZE):
                batch_data = data[i:i + BATCH_SIZE]
                self.cursor.executemany(query, batch_data)
                self.connection.commit()
                
        except Exception as e:
            print(e)
            
    def save_to_csv(self, data, file_name, path):
        
        '''
        arguments: data, file_name, path
        This method will save the data to csv file
        '''
        try:
            # create the folder if it does not exist
            if not os.path.exists(path):
                os.makedirs(path)

            # specify the full file path
            full_file_path = os.path.join(path, file_name)

            # ssave the data to a CSV file....
            data.to_csv(full_file_path, index=False)
            print(f"File '{file_name}' saved to disk in the '{path}' folder.")
        
        except Exception as e:
            print(e)
        
        
    def close(self):
        self.connection.close()
        print("close connection")

In [229]:
sql_executer = DatabaseConnection(
    host = os.getenv("HOST"),
    user = os.getenv("USER"),
    password = os.getenv("PASSWORD"),
    database = os.getenv("DATABASE")
)

In [230]:

data = sql_executer.execute_query_to_dataframe(
    query=os.getenv("QUERY"))


In [231]:
data.head()

,index,id,first_name,last_name,email,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score
0,0,1,Paul,Casey,paul.casey.1@gslingacademy.com,male,0,3,0,27,Lawyer,73,81,93,97,63,80,87
1,1,2,Danielle,Sandoval,danielle.sandoval.2@gslingacademy.com,female,0,2,0,47,Doctor,90,86,96,100,90,88,90
2,2,3,Tina,Andrews,tina.andrews.3@gslingacademy.com,female,0,9,1,13,Government Officer,81,97,95,96,65,77,94
3,3,4,Tara,Clark,tara.clark.4@gslingacademy.com,female,0,5,0,3,Artist,71,74,88,80,89,63,86
4,4,5,Anthony,Campos,anthony.campos.5@gslingacademy.com,male,0,5,0,10,Unknown,84,77,65,65,80,74,76


In [232]:
data.columns

Index(['index', 'id', 'first_name', 'last_name', 'email', 'gender',
       'part_time_job', 'absence_days', 'extracurricular_activities',
       'weekly_self_study_hours', 'career_aspiration', 'math_score',
       'history_score', 'physics_score', 'chemistry_score', 'biology_score',
       'english_score', 'geography_score'],
      dtype='object')

In [233]:
data.columns[0]

'index'

In [234]:
data[['id', 'first_name', 'last_name', 'email', 'gender',
    'part_time_job', 'absence_days', 'extracurricular_activities',
    'weekly_self_study_hours', 'career_aspiration', 'math_score',
    'history_score', 'physics_score', 'chemistry_score', 'biology_score',
    'english_score', 'geography_score']]

,id,first_name,last_name,email,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score
0,1,Paul,Casey,paul.casey.1@gslingacademy.com,male,0,3,0,27,Lawyer,73,81,93,97,63,80,87
1,2,Danielle,Sandoval,danielle.sandoval.2@gslingacademy.com,female,0,2,0,47,Doctor,90,86,96,100,90,88,90
2,3,Tina,Andrews,tina.andrews.3@gslingacademy.com,female,0,9,1,13,Government Officer,81,97,95,96,65,77,94
3,4,Tara,Clark,tara.clark.4@gslingacademy.com,female,0,5,0,3,Artist,71,74,88,80,89,63,86
4,5,Anthony,Campos,anthony.campos.5@gslingacademy.com,male,0,5,0,10,Unknown,84,77,65,65,80,74,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,Alan,Reynolds,alan.reynolds.1996@gslingacademy.com,male,0,2,0,30,Construction Engineer,83,77,84,73,75,84,82
1996,1997,Thomas,Gilbert,thomas.gilbert.1997@gslingacademy.com,male,0,2,0,20,Software Engineer,89,65,73,80,87,67,73
1997,1998,Madison,Cross,madison.cross.1998@gslingacademy.com,female,0,5,0,14,Software Engineer,97,85,63,93,68,94,78
1998,1999,Brittany,Compton,brittany.compton.1999@gslingacademy.com,female,1,10,1,5,Business Owner,51,96,72,89,95,88,75


In [235]:
data.duplicated().sum()

0

In [236]:
data.drop(data.columns[0],axis=1)

,id,first_name,last_name,email,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score
0,1,Paul,Casey,paul.casey.1@gslingacademy.com,male,0,3,0,27,Lawyer,73,81,93,97,63,80,87
1,2,Danielle,Sandoval,danielle.sandoval.2@gslingacademy.com,female,0,2,0,47,Doctor,90,86,96,100,90,88,90
2,3,Tina,Andrews,tina.andrews.3@gslingacademy.com,female,0,9,1,13,Government Officer,81,97,95,96,65,77,94
3,4,Tara,Clark,tara.clark.4@gslingacademy.com,female,0,5,0,3,Artist,71,74,88,80,89,63,86
4,5,Anthony,Campos,anthony.campos.5@gslingacademy.com,male,0,5,0,10,Unknown,84,77,65,65,80,74,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,Alan,Reynolds,alan.reynolds.1996@gslingacademy.com,male,0,2,0,30,Construction Engineer,83,77,84,73,75,84,82
1996,1997,Thomas,Gilbert,thomas.gilbert.1997@gslingacademy.com,male,0,2,0,20,Software Engineer,89,65,73,80,87,67,73
1997,1998,Madison,Cross,madison.cross.1998@gslingacademy.com,female,0,5,0,14,Software Engineer,97,85,63,93,68,94,78
1998,1999,Brittany,Compton,brittany.compton.1999@gslingacademy.com,female,1,10,1,5,Business Owner,51,96,72,89,95,88,75


In [237]:
data

,index,id,first_name,last_name,email,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score
0,0,1,Paul,Casey,paul.casey.1@gslingacademy.com,male,0,3,0,27,Lawyer,73,81,93,97,63,80,87
1,1,2,Danielle,Sandoval,danielle.sandoval.2@gslingacademy.com,female,0,2,0,47,Doctor,90,86,96,100,90,88,90
2,2,3,Tina,Andrews,tina.andrews.3@gslingacademy.com,female,0,9,1,13,Government Officer,81,97,95,96,65,77,94
3,3,4,Tara,Clark,tara.clark.4@gslingacademy.com,female,0,5,0,3,Artist,71,74,88,80,89,63,86
4,4,5,Anthony,Campos,anthony.campos.5@gslingacademy.com,male,0,5,0,10,Unknown,84,77,65,65,80,74,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,1996,Alan,Reynolds,alan.reynolds.1996@gslingacademy.com,male,0,2,0,30,Construction Engineer,83,77,84,73,75,84,82
1996,1996,1997,Thomas,Gilbert,thomas.gilbert.1997@gslingacademy.com,male,0,2,0,20,Software Engineer,89,65,73,80,87,67,73
1997,1997,1998,Madison,Cross,madison.cross.1998@gslingacademy.com,female,0,5,0,14,Software Engineer,97,85,63,93,68,94,78
1998,1998,1999,Brittany,Compton,brittany.compton.1999@gslingacademy.com,female,1,10,1,5,Business Owner,51,96,72,89,95,88,75


In [238]:
data["career_aspiration"].value_counts()

career_aspiration
Software Engineer        315
Business Owner           309
Unknown                  223
Banker                   169
Lawyer                   138
Accountant               126
Doctor                   119
Real Estate Developer     83
Stock Investor            73
Construction Engineer     68
Artist                    67
Game Developer            63
Government Officer        61
Teacher                   59
Designer                  56
Scientist                 39
Writer                    32
Name: count, dtype: int64

In [239]:
data["career_aspiration"].str.replace("Unknown","Software Engineer")

0                      Lawyer
1                      Doctor
2          Government Officer
3                      Artist
4           Software Engineer
                ...          
1995    Construction Engineer
1996        Software Engineer
1997        Software Engineer
1998           Business Owner
1999               Accountant
Name: career_aspiration, Length: 2000, dtype: object

In [240]:
# math_score','history_score', 'physics_score', 'chemistry_score', 'biology_score','english_score', 'geography_score'
data["math_score"] + data["history_score"] + data["physics_score"] + data["chemistry_score"] + data["biology_score"] +data["english_score"] + data["geography_score"]

0       574
1       640
2       605
3       551
4       521
       ... 
1995    558
1996    534
1997    578
1998    566
1999    617
Length: 2000, dtype: int64

In [241]:
data

,index,id,first_name,last_name,email,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score
0,0,1,Paul,Casey,paul.casey.1@gslingacademy.com,male,0,3,0,27,Lawyer,73,81,93,97,63,80,87
1,1,2,Danielle,Sandoval,danielle.sandoval.2@gslingacademy.com,female,0,2,0,47,Doctor,90,86,96,100,90,88,90
2,2,3,Tina,Andrews,tina.andrews.3@gslingacademy.com,female,0,9,1,13,Government Officer,81,97,95,96,65,77,94
3,3,4,Tara,Clark,tara.clark.4@gslingacademy.com,female,0,5,0,3,Artist,71,74,88,80,89,63,86
4,4,5,Anthony,Campos,anthony.campos.5@gslingacademy.com,male,0,5,0,10,Unknown,84,77,65,65,80,74,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,1996,Alan,Reynolds,alan.reynolds.1996@gslingacademy.com,male,0,2,0,30,Construction Engineer,83,77,84,73,75,84,82
1996,1996,1997,Thomas,Gilbert,thomas.gilbert.1997@gslingacademy.com,male,0,2,0,20,Software Engineer,89,65,73,80,87,67,73
1997,1997,1998,Madison,Cross,madison.cross.1998@gslingacademy.com,female,0,5,0,14,Software Engineer,97,85,63,93,68,94,78
1998,1998,1999,Brittany,Compton,brittany.compton.1999@gslingacademy.com,female,1,10,1,5,Business Owner,51,96,72,89,95,88,75


In [242]:
class DataTransformation:
    
    def __init__(self):
        pass
    def data_transformation(self,data):
        
        '''
        This method will take data as input and return transformed data
        
        arguments: data
        
        return: transformed data
        '''
        
        try:
            #drop index column from data
            data.drop(data.columns[0], axis=1,inplace=True)
            
            # replacr unknown feature with most frequent value Software Engineer
            data["career_aspiration"] = data["career_aspiration"].str.replace("Unknown","Software Engineer")
            
            # geting total score of each student
            data["total_score"] = data["math_score"] + data["history_score"] + data["physics_score"] + data["chemistry_score"] + data["biology_score"] +data["english_score"] + data["geography_score"]
            
            return data
        
        except Exception as e:
            print(e)
            
            
    def save_to_csv(self, data, file_name, path):
        
        '''
        arguments: data, file_name, path
        This method will save the data to csv file
        '''
        try:
            # create the folder if it does not exist
            if not os.path.exists(path):
                os.makedirs(path)

            # specify the full file path
            full_file_path = os.path.join(path, file_name)

            # ssave the data to a CSV file....
            data.to_csv(full_file_path, index=False)
            print(f"File '{file_name}' saved to disk in the '{path}' folder.")
        
        except Exception as e:
            print(e)
        

In [243]:
dt = DataTransformation()

In [244]:
transform_data = dt.data_transformation(data)

In [245]:
transform_data

,id,first_name,last_name,email,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score,total_score
0,1,Paul,Casey,paul.casey.1@gslingacademy.com,male,0,3,0,27,Lawyer,73,81,93,97,63,80,87,574
1,2,Danielle,Sandoval,danielle.sandoval.2@gslingacademy.com,female,0,2,0,47,Doctor,90,86,96,100,90,88,90,640
2,3,Tina,Andrews,tina.andrews.3@gslingacademy.com,female,0,9,1,13,Government Officer,81,97,95,96,65,77,94,605
3,4,Tara,Clark,tara.clark.4@gslingacademy.com,female,0,5,0,3,Artist,71,74,88,80,89,63,86,551
4,5,Anthony,Campos,anthony.campos.5@gslingacademy.com,male,0,5,0,10,Software Engineer,84,77,65,65,80,74,76,521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,Alan,Reynolds,alan.reynolds.1996@gslingacademy.com,male,0,2,0,30,Construction Engineer,83,77,84,73,75,84,82,558
1996,1997,Thomas,Gilbert,thomas.gilbert.1997@gslingacademy.com,male,0,2,0,20,Software Engineer,89,65,73,80,87,67,73,534
1997,1998,Madison,Cross,madison.cross.1998@gslingacademy.com,female,0,5,0,14,Software Engineer,97,85,63,93,68,94,78,578
1998,1999,Brittany,Compton,brittany.compton.1999@gslingacademy.com,female,1,10,1,5,Business Owner,51,96,72,89,95,88,75,566


In [246]:
dt.save_to_csv(
    data = transform_data,
    file_name = "transform_student_data.csv",
    path = "data/")

File 'transform_student_data.csv' saved to disk in the 'data/' folder.


In [247]:
# def execute_query_to_dataframe(query, connection):
#     """
#     Executes a SQL query and returns the results as a pandas DataFrame.

#     Args:
#         query (str): SQL query to execute.
#         connection (pymysql.connections.Connection): Established database connection.

#     Returns:
#         pd.DataFrame: DataFrame containing the query results.
#     """
#     try:
#         df = pd.read_sql_query(query, connection)
#         return df

#     except Exception as e:
#         raise CustomException(f"Error executing query: {str(e)}")

In [248]:
class AWS_S3Manager:

    def __init__(self, bucket_name, aws_access_key_id, aws_secret_access_key, region_name):
        self.bucket_name = bucket_name
        self.s3_client = boto3.client(
            service_name='s3',
            region_name=region_name,
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key
        )

    def upload_file(self, local_filename, s3_key):
        
        '''
        arguments: local_filename, s3_key
        
        Upload file to S3_bucket 
        ''' 
        try:
            self.s3_client.upload_file(local_filename, self.bucket_name, s3_key)
            print(f"File '{local_filename}' uploaded to S3 bucket as '{s3_key}'.")

        except Exception as e:
            print(e)
        
        
    def download_file(self, s3_key, local_filename, target_directory="downloads"):
        
        '''
        arguments: local_filename, target_directory,s3_key
        
        Downloading the file to the target directory specified   in the S3_bucket configuration file and 
        rename the file to the local_filename specified in the S3_bucket configuration file.
        '''
        try:

            if not os.path.exists(target_directory):
                os.makedirs(target_directory)# create the target directory if it doesn't exist

            # Construct the full local path
            local_path = os.path.join(target_directory, local_filename)

            # Download the file
            self.s3_client.download_file(self.bucket_name, s3_key, local_path)
            print(f"File '{s3_key}' downloaded from S3 bucket and saved as '{local_path}'.")

        except Exception as e:
            print(e)

    def read_csv_from_s3(self, s3_key):
        
        '''
        arguments: s3_key
        
        Reads CSV file from S3 bucket.
        
        returns: data frame 
        '''
        try:
        
            obj = self.s3_client.get_object(Bucket=self.bucket_name, Key=s3_key)
            df = pd.read_csv(obj['Body'])
            return df
        
        except Exception as e:
            print(e)

In [249]:
aws = AWS_S3Manager(
    bucket_name = os.getenv("BUCKET_NAME"),
    aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name = os.getenv("REGION_NAME")
)

In [250]:
aws.upload_file(
    local_filename = "data/transform_student_data.csv",
    s3_key = "transform_student_data.csv"
)

File 'data/transform_student_data.csv' uploaded to S3 bucket as 'transform_student_data.csv'.


In [251]:
data_s3 = aws.read_csv_from_s3(s3_key="transform_student_data.csv")

In [252]:
# convert in to tuple to insert in database 
data_s3 = list(data_s3.itertuples(index=False,name=None))

In [253]:
# create 2 table ym_transform_students_data in mysql data base 
sql_executer.execute_query("""
    CREATE TABLE IF NOT EXISTS ym_transform_students_data (
        id BIGINT NOT NULL,
        first_name VARCHAR(255) NOT NULL,
        last_name VARCHAR(255) NOT NULL,
        email VARCHAR(255) NOT NULL,
        gender VARCHAR(255) NOT NULL,
        part_time_job BIGINT NOT NULL,
        absence_days BIGINT NOT NULL,
        extracurricular_activities BIGINT NOT NULL,
        weekly_self_study_hours BIGINT NOT NULL,
        career_aspiration BIGINT NOT NULL,
        math_score BIGINT NOT NULL,
        history_score BIGINT NOT NULL,
        physics_score BIGINT NOT NULL,
        chemistry_score BIGINT NOT NULL,
        biology_score BIGINT NOT NULL,
        english_score BIGINT NOT NULL,
        geography_score BIGINT NOT NULL,
        total_score  BIGINT NOT NULL);
    """)

[]

In [254]:
insert_qury = """
    INSERT INTO ym_transform_students_data (
        id,
        first_name,
        last_name,
        email,
        gender,
        part_time_job,
        absence_days,
        extracurricular_activities,
        weekly_self_study_hours,
        career_aspiration,
        math_score,
        history_score,
        physics_score,
        chemistry_score,
        biology_score,
        english_score,
        geography_score,
        total_score)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    
sql_executer.insert_data_batch(
        insert_qury,
        data_s3,
        BATCH_SIZE= 200
    )

[(1,
  'Paul',
  'Casey',
  'paul.casey.1@gslingacademy.com',
  'male',
  0,
  3,
  0,
  27,
  'Lawyer',
  73,
  81,
  93,
  97,
  63,
  80,
  87,
  574),
 (2,
  'Danielle',
  'Sandoval',
  'danielle.sandoval.2@gslingacademy.com',
  'female',
  0,
  2,
  0,
  47,
  'Doctor',
  90,
  86,
  96,
  100,
  90,
  88,
  90,
  640),
 (3,
  'Tina',
  'Andrews',
  'tina.andrews.3@gslingacademy.com',
  'female',
  0,
  9,
  1,
  13,
  'Government Officer',
  81,
  97,
  95,
  96,
  65,
  77,
  94,
  605),
 (4,
  'Tara',
  'Clark',
  'tara.clark.4@gslingacademy.com',
  'female',
  0,
  5,
  0,
  3,
  'Artist',
  71,
  74,
  88,
  80,
  89,
  63,
  86,
  551),
 (5,
  'Anthony',
  'Campos',
  'anthony.campos.5@gslingacademy.com',
  'male',
  0,
  5,
  0,
  10,
  'Software Engineer',
  84,
  77,
  65,
  65,
  80,
  74,
  76,
  521),
 (6,
  'Kelly',
  'Wade',
  'kelly.wade.6@gslingacademy.com',
  'female',
  0,
  2,
  0,
  26,
  'Software Engineer',
  93,
  100,
  67,
  78,
  72,
  80,
  84,
  574),
 